In [1]:
import jax
jax.config.update("jax_enable_x64", True)

In [2]:
import scvi
import scanpy as sc
import os
import juniper

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex"
adata_train = sc.read(os.path.join(output_dir, "adata_train_biolord_split_30.h5ad"))
adata_test = sc.read(os.path.join(output_dir, "adata_test_biolord_split_30.h5ad"))
adata_ood = sc.read(os.path.join(output_dir, "adata_ood_biolord_split_30.h5ad")) 


/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [23]:
juniper.latent.model.FactorVI.setup_anndata(adata_train)
vae = juniper.latent.model.FactorVI(adata_train, count_input=False, n_latent = 10, encoder_layers= (2048, 2048, 2048), decoder_layers = (2048, 2048, 2048), disentangling_weight=0.1, kld_weight=0.5)


/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


In [24]:
vae.train(
    max_epochs=2000,
    batch_size=1024,
    plan_kwargs=dict(
        lr=1e-4,
    ),
    early_stopping=True,
    early_stopping_patience=500,
)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/ot_pert_geno ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/ot_pert_geno ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/ictstr01/home/icb/dominik.klein/mambaforge/envs/ot_pe

Epoch 590/2000:  30%|██▉       | 590/2000 [1:13:18<2:55:12,  7.46s/it, v_num=1, train_loss=30.3, rec_loss=27.1, kld_loss=6.51]
Monitored metric reconstruction_loss_validation did not improve in the last 500 records. Best score: 25.688. Signaling Trainer to stop.


In [25]:
adata_train.obsm["X_scVI"] = vae.get_latent_representation(adata_train, give_mean=True)

In [26]:
adata_train.obsm["reconstruction"] = vae.get_reconstructed_expression(adata_train, give_mean=True)

In [27]:
from sklearn.metrics import pairwise_distances, r2_score
import numpy as np

In [28]:
r2_score(np.mean(adata_train.obsm["reconstruction"], axis=0), np.array(np.mean(adata_train.X, axis=0))[0,:])

0.9950848577258933

In [29]:
adata_ood.obsm["X_scVI"] = vae.get_latent_representation(adata_ood, give_mean=True)
adata_ood.obsm["reconstruction"] = vae.get_reconstructed_expression(adata_ood, give_mean=True)
r2_score(np.mean(adata_ood.obsm["reconstruction"], axis=0), np.array(np.mean(adata_ood.X, axis=0))[0,:])

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


0.9240844114443761

In [30]:
adata_test.obsm["X_scVI"] = vae.get_latent_representation(adata_test, give_mean=True)
adata_test.obsm["reconstruction"] = vae.get_reconstructed_expression(adata_test, give_mean=True)
r2_score(np.mean(adata_test.obsm["reconstruction"], axis=0), np.array(np.mean(adata_test.X, axis=0))[0,:])

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


0.9953748657941697

In [12]:
save_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/sciplex_factorvi_test_10"
vae.save(save_path, overwrite=True)

In [13]:
adata_ood.uns['rank_genes_groups_cov_all'] = adata_train.uns['rank_genes_groups_cov_all']

In [14]:
adata_train.write(os.path.join(output_dir, "adata_train_biolord_split_30_with_scvi_10.h5ad"))
adata_test.write(os.path.join(output_dir, "adata_test_biolord_split_30_with_scvi_10.h5ad"))
adata_ood.write(os.path.join(output_dir, "adata_ood_biolord_split_30_with_scvi_10.h5ad")) 
